# SeaWIFS

In [15]:
import requests
import os

def download_file(url, filename):
    if os.path.exists(filename):
        print(f"File '{filename}' already exists. Skipping download.")
        return

    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

# make sure 'data/' directory exists
if not os.path.exists('data/'):
    os.makedirs('data/')

base_url = 'http://orca.science.oregonstate.edu/data/1x2/monthly/par.r2014.seawifs/hdf/par.s.{}.tar'

for year in range(1997, 2011):  # 2011 is not included
    url = base_url.format(year)
    filename = f'data/par.s.{year}.tar'
    download_file(url, filename)


File 'data/par.s.1997.tar' already exists. Skipping download.
File 'data/par.s.1998.tar' already exists. Skipping download.
File 'data/par.s.1999.tar' already exists. Skipping download.
File 'data/par.s.2000.tar' already exists. Skipping download.
File 'data/par.s.2001.tar' already exists. Skipping download.
File 'data/par.s.2002.tar' already exists. Skipping download.
File 'data/par.s.2003.tar' already exists. Skipping download.
File 'data/par.s.2004.tar' already exists. Skipping download.
File 'data/par.s.2005.tar' already exists. Skipping download.
File 'data/par.s.2006.tar' already exists. Skipping download.
File 'data/par.s.2007.tar' already exists. Skipping download.
File 'data/par.s.2008.tar' already exists. Skipping download.
File 'data/par.s.2009.tar' already exists. Skipping download.
File 'data/par.s.2010.tar' already exists. Skipping download.


## Step two: calculate the mean of whole data

In [3]:
conda update numpy

Retrieving notices: ...working... ERROR conda.notices.fetch:get_channel_notice_response(67): Request error <HTTPSConnectionPool(host='repo.anaconda.com', port=443): Max retries exceeded with url: /pkgs/main/notices.json (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 403 Forbidden')))> for channel: defaults url: https://repo.anaconda.com/pkgs/main/notices.json
ERROR conda.notices.fetch:get_channel_notice_response(67): Request error <HTTPSConnectionPool(host='repo.anaconda.com', port=443): Max retries exceeded with url: /pkgs/r/notices.json (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 403 Forbidden')))> for channel: defaults url: https://repo.anaconda.com/pkgs/r/notices.json
done

ProxyError: Conda cannot proceed due to an error in your proxy configuration.
Check for typos and other configuration errors in any '.netrc' file in your home directory,
any environment variables ending in '_PROXY', and any other syst

In [4]:
conda update pyhdf


ProxyError: Conda cannot proceed due to an error in your proxy configuration.
Check for typos and other configuration errors in any '.netrc' file in your home directory,
any environment variables ending in '_PROXY', and any other system-wide proxy
configuration settings.



Note: you may need to restart the kernel to use updated packages.


In [29]:
import os
import tarfile
import gzip
import tempfile
import pandas as pd
import numpy as np
from pyhdf.SD import SD, SDC

# Define the latitude and longitude arrays
lats = np.linspace(90, -90, 1080)
lons = np.linspace(-180, 180, 2160)

# Create a meshgrid of latitudes and longitudes
lon_grid, lat_grid = np.meshgrid(lons, lats, indexing='ij')

# Flatten the arrays for dataframe construction
lat_flat = lat_grid.flatten()
lon_flat = lon_grid.flatten()

# Create a multi-index from the latitude and longitude arrays
index = pd.MultiIndex.from_arrays([lat_flat, lon_flat], names=['Lat', 'Long'])

# Create an empty DataFrame with this index
df = pd.DataFrame(index=index)

# Loop through each tar file
for year in range(1997, 2011):
    # Open the tar file
    tar_filename = f'data/par.s.{year}.tar'
    with tarfile.open(tar_filename, "r:") as tar:

         # Create an empty DataFrame to store data for this year
        temp_df = pd.DataFrame(index=index)

        # Loop through each member of the tar file
        for member in tar.getmembers():
            # If it's a .hdf.gz file
            if member.name.endswith('.hdf.gz'):
                # Open the .hdf.gz file
                f = tar.extractfile(member)
                with gzip.open(f, 'rb') as gz:
                    # Decompress the .hdf.gz file to a temporary file
                    with tempfile.NamedTemporaryFile() as tmp:
                        tmp.write(gz.read())
                        tmp.seek(0)  # Go back to the start of the file

                        # Open the temporary .hdf file
                        hdf_file = SD(tmp.name, SDC.READ)

                        # Access the 'npp' dataset
                        data = hdf_file.select('par')[:]

                        # Replace '-9999.0' with NaN
                        data[data == -9999.0] = np.nan

                        # Flatten the data and add it to the temporary DataFrame
                        data_flat = data.flatten()
                        temp_df[member.name] = data_flat

                        # Close the file
                        hdf_file.end()

        # After going through all files for the year, calculate the mean for each location (ignoring NaNs)
        mean_data = temp_df.mean(axis=1, skipna=True)

        # Add this DataFrame to the main DataFrame
        df[str(year)] = mean_data

# Reset the index of the DataFrame, making 'Lat' and 'Long' normal columns
df.reset_index(inplace=True)
                      

# Save the DataFrame to a CSV file
df.to_csv('par_means_s.csv', index=False)


In [45]:
df_par_means = pd.read_csv('par_means_s.csv')
df_par_means.head()

,Lat,Long,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010
0,90.000000,-180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,89.833179,-180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,89.666358,-180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,89.499537,-180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,89.332715,-180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df_par_means.describe()

,Lat,Long,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010
count,2.332800e+06,2.332800e+06,1.328358e+06,1.431556e+06,1.434942e+06,1.433280e+06,1.428070e+06,1.525700e+06,1.559677e+06,1.564153e+06,1.571626e+06,1.570545e+06,1.587869e+06,1.579729e+06,1.573119e+06,1.581705e+06
mean,1.347462e-14,5.988442e-16,3.648680e+01,3.293971e+01,3.286141e+01,3.272164e+01,3.257054e+01,3.404737e+01,3.359226e+01,3.340154e+01,3.317612e+01,3.329809e+01,3.287351e+01,3.321420e+01,3.320171e+01,3.301338e+01
std,5.200967e+01,1.039712e+02,1.437401e+01,1.081675e+01,1.097235e+01,1.110469e+01,1.137511e+01,1.146245e+01,1.167125e+01,1.173296e+01,1.186311e+01,1.179777e+01,1.205750e+01,1.193083e+01,1.184485e+01,1.192258e+01
min,-9.000000e+01,-1.800000e+02,1.999000e-03,1.199300e-02,2.119980e-01,2.999900e-02,1.199300e-02,3.266356e-02,5.799689e-02,7.399689e-02,6.799689e-02,7.899690e-02,4.599689e-02,1.459969e-01,1.899689e-02,9.199689e-02
25%,-4.500000e+01,-9.000000e+01,2.955235e+01,2.242377e+01,2.230057e+01,2.197562e+01,2.159606e+01,2.379849e+01,2.268313e+01,2.230756e+01,2.221823e+01,2.210608e+01,2.171053e+01,2.209200e+01,2.220668e+01,2.207667e+01
50%,7.105427e-15,-1.425249e-14,3.971651e+01,3.352026e+01,3.345121e+01,3.349470e+01,3.371332e+01,3.578067e+01,3.460883e+01,3.459970e+01,3.409866e+01,3.417763e+01,3.401879e+01,3.436450e+01,3.401798e+01,3.408301e+01
75%,4.500000e+01,9.000000e+01,4.783021e+01,4.351616e+01,4.318112e+01,4.327298e+01,4.326261e+01,4.390462e+01,4.435869e+01,4.429033e+01,4.407848e+01,4.423293e+01,4.395175e+01,4.411874e+01,4.414447e+01,4.400641e+01
max,9.000000e+01,1.800000e+02,6.624800e+01,6.409000e+01,6.307400e+01,6.310400e+01,6.422200e+01,6.720000e+01,6.812823e+01,6.927200e+01,6.789000e+01,6.712600e+01,6.914400e+01,6.941600e+01,6.787400e+01,6.725800e+01


## Step three: read cleaned excel

In [32]:
# read new data
# read each page from excel file
import pandas as pd
import numpy as np
# find the path of the file called 'cleaned.xlsx'
filename='cleaned'
Pacific=pd.read_excel(filename+'.xlsx',sheet_name='Pacific')
Atlantic=pd.read_excel(filename+'.xlsx',sheet_name='Atlantic')
Mediterranean=pd.read_excel(filename+'.xlsx',sheet_name='Mediterranean')
Southern=pd.read_excel(filename+'.xlsx',sheet_name='Southern Ocean')
Arctic=pd.read_excel(filename+'.xlsx',sheet_name='Arctic')

## Step four: Melt for merge

In [33]:
df_par_melt = df_par_means.melt(id_vars=['Lat', 'Long'], var_name='Year', value_name='mld')

## Step five: Select decimal

In [34]:
# Then merge on these columns
# assuming df_melted_variable1-5 and ocean_dfs are your dataframes
ocean_names = ["Pacific", "Atlantic", "Mediterranean", "Southern", "Arctic"]
# change this to your dataframes
df_melted_list = [df_par_melt]
ocean_dfs = [Pacific, Atlantic, Mediterranean, Southern, Arctic]


for df in df_melted_list + ocean_dfs:
    # convert types safely
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce').astype('Int64')
    df['Lat'] = pd.to_numeric(df['Lat'], errors='coerce')
    df['Long'] = pd.to_numeric(df['Long'], errors='coerce')


# round lat and long to 0.0001
# I believe round is a way of thresholding the data, let me know if there is a better wayof matching 
# the lat and longs
for df in df_melted_list + ocean_dfs:
    df['Lat'] = df['Lat'].round(1)
    df['Long'] = df['Long'].round(1)

In [35]:
# check if there is a match

# Create a set of tuples for the Pacific dataframe
pacific_coords = set(zip(Pacific['Lat'], Pacific['Long']))

# Create a set of tuples for the df_npp dataframe
npp_coords = set(zip(df_par_melt['Lat'], df_par_melt['Long']))

# Find the common coordinates
common_coords = pacific_coords.intersection(npp_coords)

print(f"Common coordinates between Pacific and df_npp: {len(common_coords)}")


Common coordinates between Pacific and df_npp: 16


## Step six: Merge

In [36]:
import pandas as pd
import numpy as np

merged_dfs = []  # List to store merged dataframes

for ocean_name, ocean_df in zip(ocean_names, ocean_dfs):
    # merge ocean_df with all dataframes in df_melted_list
    for df_melted in df_melted_list:
        ocean_df = pd.merge(ocean_df, df_melted, on=['Lat', 'Long', 'Year'], how='left')

    # Fill NaN values with the mean of each column
    for col in ocean_df.select_dtypes(include=[np.number]).columns:
        ocean_df[col] = ocean_df[col].astype(float)
        ocean_df[col].fillna(ocean_df[col].mean(), inplace=True)
        if ocean_df[col].apply(float.is_integer).all():  # Check if all values are integer
            ocean_df[col] = ocean_df[col].astype('Int64')  # Change dtype back to integer

    merged_dfs.append(ocean_df)  # Append the merged dataframe to the list

with pd.ExcelWriter('merged_par_s.xlsx') as writer:
    for ocean_name, merged_df in zip(ocean_names, merged_dfs):
        merged_df.to_excel(writer, sheet_name=ocean_name)  # Write each merged dataframe to a different sheet


# MODIS

In [25]:
import requests
import os

def download_file(url, filename):
    if os.path.exists(filename):
        print(f"File '{filename}' already exists. Skipping download.")
        return

    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

# make sure 'data/' directory exists
if not os.path.exists('data/'):
    os.makedirs('data/')


# download the data from MODIS:  http://orca.science.oregonstate.edu/1080.by.2160.monthly.hdf.vgpm.m.chl.m.sst.php

new_url = 'http://orca.science.oregonstate.edu/data/1x2/monthly/par.modis.r2022/hdf/par.m.{}.tar'

for year in range(2002, 2024):  # 2024 is not included
    url = new_url.format(year)
    filename = f'data/par.s.{year}.tar'
    download_file(url, filename)

File 'data/par.s.2002.tar' already exists. Skipping download.
File 'data/par.s.2003.tar' already exists. Skipping download.
File 'data/par.s.2004.tar' already exists. Skipping download.
File 'data/par.s.2005.tar' already exists. Skipping download.
File 'data/par.s.2006.tar' already exists. Skipping download.
File 'data/par.s.2007.tar' already exists. Skipping download.
File 'data/par.s.2008.tar' already exists. Skipping download.
File 'data/par.s.2009.tar' already exists. Skipping download.
File 'data/par.s.2010.tar' already exists. Skipping download.
File 'data/par.s.2011.tar' already exists. Skipping download.
File 'data/par.s.2012.tar' already exists. Skipping download.
File 'data/par.s.2013.tar' already exists. Skipping download.
File 'data/par.s.2014.tar' already exists. Skipping download.
File 'data/par.s.2015.tar' already exists. Skipping download.
File 'data/par.s.2016.tar' already exists. Skipping download.
File 'data/par.s.2017.tar' already exists. Skipping download.
File 'da

In [46]:
# Define the latitude and longitude arrays
lats = np.linspace(90, -90, 1080)
lons = np.linspace(-180, 180, 2160)

# Create a meshgrid of latitudes and longitudes
lon_grid, lat_grid = np.meshgrid(lons, lats, indexing='ij')

# Flatten the arrays for dataframe construction
lat_flat = lat_grid.flatten()
lon_flat = lon_grid.flatten()

# Create a multi-index from the latitude and longitude arrays
index = pd.MultiIndex.from_arrays([lat_flat, lon_flat], names=['Lat', 'Long'])

# Create an empty DataFrame with this index
df = pd.DataFrame(index=index)

# Loop through each tar file
for year in range(1997, 2024):
    # Open the tar file
    tar_filename = f'data/par.s.{year}.tar'
    with tarfile.open(tar_filename, "r:") as tar:

         # Create an empty DataFrame to store data for this year
        temp_df = pd.DataFrame(index=index)

        # Loop through each member of the tar file
        for member in tar.getmembers():
            # If it's a .hdf.gz file
            if member.name.endswith('.hdf.gz'):
                # Open the .hdf.gz file
                f = tar.extractfile(member)
                with gzip.open(f, 'rb') as gz:
                    # Decompress the .hdf.gz file to a temporary file
                    with tempfile.NamedTemporaryFile() as tmp:
                        tmp.write(gz.read())
                        tmp.seek(0)  # Go back to the start of the file

                        # Open the temporary .hdf file
                        hdf_file = SD(tmp.name, SDC.READ)

                        # Access the 'npp' dataset
                        data = hdf_file.select('par')[:]

                        # Replace '-9999.0' with NaN
                        data[data == -9999.0] = np.nan

                        # Flatten the data and add it to the temporary DataFrame
                        data_flat = data.flatten()
                        temp_df[member.name] = data_flat

                        # Close the file
                        hdf_file.end()

        # After going through all files for the year, calculate the mean for each location (ignoring NaNs)
        mean_data = temp_df.mean(axis=1, skipna=True)

        # Add this DataFrame to the main DataFrame
        df[str(year)] = mean_data

# Reset the index of the DataFrame, making 'Lat' and 'Long' normal columns
df.reset_index(inplace=True)
                      

# Save the DataFrame to a CSV file
df.to_csv('par_means_m.csv', index=False)


In [47]:
df_par_means_m = pd.read_csv('par_means_m.csv')
df_par_means_m.head()

,Lat,Long,1997,1998,1999,2000,2001,2002,2003,2004,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,90.000000,-180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,89.833179,-180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,89.666358,-180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,89.499537,-180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,89.332715,-180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df_par_means_m.describe()

,Lat,Long,1997,1998,1999,2000,2001,2002,2003,2004,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
count,2.332800e+06,2.332800e+06,1.328358e+06,1.431556e+06,1.434942e+06,1.433280e+06,1.428070e+06,1.525700e+06,1.559677e+06,1.564153e+06,...,1.574863e+06,1.597800e+06,1.618205e+06,1.605778e+06,1.609688e+06,1.616116e+06,1.629161e+06,1.616048e+06,1.615011e+06,1.349397e+06
mean,1.347462e-14,5.988442e-16,3.648680e+01,3.293971e+01,3.286141e+01,3.272164e+01,3.257054e+01,3.404737e+01,3.359226e+01,3.340154e+01,...,3.313329e+01,3.333278e+01,3.315810e+01,3.326634e+01,3.323867e+01,3.325779e+01,3.290469e+01,3.324599e+01,3.346675e+01,3.945527e+01
std,5.200967e+01,1.039712e+02,1.437401e+01,1.081675e+01,1.097235e+01,1.110469e+01,1.137511e+01,1.146245e+01,1.167125e+01,1.173296e+01,...,1.198258e+01,1.169108e+01,1.174541e+01,1.175228e+01,1.176685e+01,1.176853e+01,1.203324e+01,1.172796e+01,1.144901e+01,1.570208e+01
min,-9.000000e+01,-1.800000e+02,1.999000e-03,1.199300e-02,2.119980e-01,2.999900e-02,1.199300e-02,3.266356e-02,5.799689e-02,7.399689e-02,...,1.399689e-02,2.239969e-01,2.199689e-02,4.199689e-02,5.399689e-02,1.229969e-01,7.599689e-02,2.599689e-02,2.799689e-02,1.899689e-02
25%,-4.500000e+01,-9.000000e+01,2.955235e+01,2.242377e+01,2.230057e+01,2.197562e+01,2.159606e+01,2.379849e+01,2.268313e+01,2.230756e+01,...,2.216235e+01,2.247413e+01,2.246878e+01,2.246191e+01,2.224007e+01,2.242867e+01,2.205770e+01,2.243214e+01,2.318759e+01,3.223234e+01
50%,7.105427e-15,-1.425249e-14,3.971651e+01,3.352026e+01,3.345121e+01,3.349470e+01,3.371332e+01,3.578067e+01,3.460883e+01,3.459970e+01,...,3.399547e+01,3.414306e+01,3.373656e+01,3.412461e+01,3.418643e+01,3.399836e+01,3.382202e+01,3.405156e+01,3.416727e+01,4.142003e+01
75%,4.500000e+01,9.000000e+01,4.783021e+01,4.351616e+01,4.318112e+01,4.327298e+01,4.326261e+01,4.390462e+01,4.435869e+01,4.429033e+01,...,4.426788e+01,4.431431e+01,4.426306e+01,4.405948e+01,4.410228e+01,4.428291e+01,4.393111e+01,4.400210e+01,4.387196e+01,5.175900e+01
max,9.000000e+01,1.800000e+02,6.624800e+01,6.409000e+01,6.307400e+01,6.310400e+01,6.422200e+01,6.720000e+01,6.812823e+01,6.927200e+01,...,6.885600e+01,6.776150e+01,6.554400e+01,6.914800e+01,6.658200e+01,6.887600e+01,6.776000e+01,6.863000e+01,6.780400e+01,6.852000e+01


In [49]:
# read new data
# read each page from excel file
import pandas as pd
import numpy as np
# find the path of the file called 'cleaned.xlsx'
filename='cleaned'
Pacific=pd.read_excel(filename+'.xlsx',sheet_name='Pacific')
Atlantic=pd.read_excel(filename+'.xlsx',sheet_name='Atlantic')
Mediterranean=pd.read_excel(filename+'.xlsx',sheet_name='Mediterranean')
Southern=pd.read_excel(filename+'.xlsx',sheet_name='Southern Ocean')
Arctic=pd.read_excel(filename+'.xlsx',sheet_name='Arctic')

In [50]:
df_par_melt_m = df_par_means_m.melt(id_vars=['Lat', 'Long'], var_name='Year', value_name='mld')

In [51]:
# Then merge on these columns
# assuming df_melted_variable1-5 and ocean_dfs are your dataframes
ocean_names = ["Pacific", "Atlantic", "Mediterranean", "Southern", "Arctic"]
# change this to your dataframes
df_melted_list = [df_par_melt_m]
ocean_dfs = [Pacific, Atlantic, Mediterranean, Southern, Arctic]


for df in df_melted_list + ocean_dfs:
    # convert types safely
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce').astype('Int64')
    df['Lat'] = pd.to_numeric(df['Lat'], errors='coerce')
    df['Long'] = pd.to_numeric(df['Long'], errors='coerce')


# round lat and long to 0.0001
# I believe round is a way of thresholding the data, let me know if there is a better wayof matching 
# the lat and longs
for df in df_melted_list + ocean_dfs:
    df['Lat'] = df['Lat'].round(1)
    df['Long'] = df['Long'].round(1)

In [52]:
# check if there is a match

# Create a set of tuples for the Pacific dataframe
pacific_coords = set(zip(Pacific['Lat'], Pacific['Long']))

# Create a set of tuples for the df_npp dataframe
npp_coords = set(zip(df_par_melt_m['Lat'], df_par_melt_m['Long']))

# Find the common coordinates
common_coords = pacific_coords.intersection(npp_coords)

print(f"Common coordinates between Pacific and df_npp: {len(common_coords)}")


Common coordinates between Pacific and df_npp: 16


In [53]:
import pandas as pd
import numpy as np

merged_dfs = []  # List to store merged dataframes

for ocean_name, ocean_df in zip(ocean_names, ocean_dfs):
    # merge ocean_df with all dataframes in df_melted_list
    for df_melted in df_melted_list:
        ocean_df = pd.merge(ocean_df, df_melted, on=['Lat', 'Long', 'Year'], how='left')

    # Fill NaN values with the mean of each column
    for col in ocean_df.select_dtypes(include=[np.number]).columns:
        ocean_df[col] = ocean_df[col].astype(float)
        ocean_df[col].fillna(ocean_df[col].mean(), inplace=True)
        if ocean_df[col].apply(float.is_integer).all():  # Check if all values are integer
            ocean_df[col] = ocean_df[col].astype('Int64')  # Change dtype back to integer

    merged_dfs.append(ocean_df)  # Append the merged dataframe to the list

with pd.ExcelWriter('merged_par_m.xlsx') as writer:
    for ocean_name, merged_df in zip(ocean_names, merged_dfs):
        merged_df.to_excel(writer, sheet_name=ocean_name)  # Write each merged dataframe to a different sheet
